In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.models import Model

# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 1000

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(columns="salary_in_usd")  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer = LSTM(128, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
lstm_layer2 = LSTM(64)(dropout_layer)
output_layer = Dense(1, activation='linear')(lstm_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


Mean Squared Error: 1940.6839011141424
Root Mean Squared Error: 44.05319399446699

In [ ]:
df.head()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Drop the "salary" column
# df = df.drop(columns=["salary"])

# Split data into features and target variable
X = df.drop(columns=["salary_in_usd","salary"])  # Features
y = df["salary_in_usd"]  # Target variable

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess numerical data
numerical_columns = X.select_dtypes(include=np.number).columns
scaler = StandardScaler()
X_train_numerical_scaled = scaler.fit_transform(X_train[numerical_columns])
X_test_numerical_scaled = scaler.transform(X_test[numerical_columns])

# Preprocess text data
max_words = 10000  # Maximum number of words to tokenize
max_sequence_length = 100  # Maximum sequence length for padding

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train.select_dtypes(include=[object]).values.flatten())

X_train_text_seq = tokenizer.texts_to_sequences(X_train.select_dtypes(include=[object]).values.flatten())
X_train_text_seq = pad_sequences(X_train_text_seq, maxlen=max_sequence_length)

X_test_text_seq = tokenizer.texts_to_sequences(X_test.select_dtypes(include=[object]).values.flatten())
X_test_text_seq = pad_sequences(X_test_text_seq, maxlen=max_sequence_length)

# Define the model architecture
model = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_sequence_length),
    SpatialDropout1D(0.2),
    LSTM(100),
    Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Print model summary
print("Model summary:")
print(model.summary())

# Train the model
history = model.fit([X_train_text_seq, X_train_numerical_scaled], y_train, epochs=20, batch_size=64, validation_data=([X_test_text_seq, X_test_numerical_scaled], y_test), callbacks=[EarlyStopping(patience=3)])

# Evaluate the model
y_pred = model.predict([X_test_text_seq, X_test_numerical_scaled])
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the data
data = pd.read_csv('jobs_in_data.csv')

# Divide the salary_in_usd column by 1000
data['salary_in_usd'] /= 1000

# Exclude non-numeric columns
numeric_data = data.select_dtypes(include=[np.number])

# Prepare data for LSTM
X = numeric_data.drop(columns=['salary_in_usd', 'salary']).values
y = numeric_data['salary_in_usd'].values

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for LSTM input (samples, time steps, features)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))

Epoch 1/200
188/188 [==============================] - 2s 3ms/step - loss: 26203.0371 - val_loss: 25959.7969
Epoch 2/200
188/188 [==============================] - 1s 3ms/step - loss: 24963.1914 - val_loss: 23699.5000
Epoch 3/200
188/188 [==============================] - 1s 3ms/step - loss: 21624.4883 - val_loss: 19336.3613
Epoch 4/200
188/188 [==============================] - 1s 3ms/step - loss: 16822.4434 - val_loss: 14270.3262
Epoch 5/200
188/188 [==============================] - 1s 3ms/step - loss: 12033.0127 - val_loss: 9857.4814
Epoch 6/200
188/188 [==============================] - 0s 2ms/step - loss: 8268.8379 - val_loss: 6735.5713
Epoch 7/200
188/188 [==============================] - 0s 2ms/step - loss: 5878.9907 - val_loss: 4950.9014
Epoch 8/200
188/188 [==============================] - 1s 3ms/step - loss: 4630.8750 - val_loss: 4127.0215
Epoch 9/200
188/188 [==============================] - 1s 3ms/step - loss: 4114.7617 - val_loss: 3828.0190
Epoch 10/200
188/188 [======